In [ ]:
# DO NOT REMOVE

# Catalogue pre-processing
## Settings

In [ ]:
%matplotlib inline

import os
import re
import sys
import h5py
import numpy
import subprocess
try:
    import cPickle as pickle
except: 
    import pickle

import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

from mpl_toolkits.basemap import Basemap

from copy import deepcopy
from oqmbt.oqt_project import OQtProject, OQtModel, OQtSource

from openquake.hmtk.seismicity.occurrence.weichert import Weichert
from openquake.hmtk.seismicity.completeness.comp_stepp_1971 import Stepp1971 
from openquake.hmtk.plotting.seismicity.completeness.plot_stepp_1972 import create_stepp_plot
from openquake.hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_trunc_gr_model
from openquake.hmtk.plotting.seismicity.catalogue_plots import (plot_magnitude_time_scatter, 
                                                      plot_magnitude_time_density)
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector
from openquake.hmtk.seismicity.declusterer.dec_gardner_knopoff import GardnerKnopoffType1
from openquake.hmtk.seismicity.declusterer.distance_time_windows import GardnerKnopoffWindow

In [ ]:
#
# load project information 
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
oqtkp.directory = os.path.dirname(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
catalogue_csv_filename = os.path.join(oqtkp.directory, getattr(model,'catalogue_csv_filename'))
#
# info
text_length = 30
print ('{:s}: {:s}'.format('Model id'.ljust(text_length), model_id))
print ('{:s}: {:s}'.format('Reading'.ljust(text_length), catalogue_csv_filename))

In [ ]:
#
# cutoff magnitude is the lowest magnitude treshold. All the events below this threshold 
# are filtered out at the beginning of the processing workflow
cutoff_magnitude = float(model.catalogue_cutoff_magnitude)
#
# width of bins used to discretise the MFDs
binwidth = float(model.mfd_binwidth)
#
# printing info
print ('{:s}: {:.2f}'.format('MFD bin width'.ljust(text_length), binwidth))
print ('{:s}: {:.2f}'.format('Magnitude cutoff'.ljust(text_length), cutoff_magnitude))

## Earthquake catalogue

In [ ]:
#
# parsing the .csv catalogue
catalogue_parser = CsvCatalogueParser(catalogue_csv_filename)
catalogue = catalogue_parser.read_file()
selector = CatalogueSelector(catalogue, create_copy=False)
selector.within_magnitude_range(cutoff_magnitude, 10.)
dep_max = float(model.catalogue_maximum_depth)
dep_min = float(model.catalogue_minimum_depth)
tmp = selector.within_depth_range(dep_max, dep_min)
#
# info
print ('Catalogue:')
print ('    minimum hypocentral depth: {:.2f}'.format(numpy.min(catalogue.data['depth'])))
print ('    maximum hypocentral depth: {:.2f}'.format(numpy.max(catalogue.data['depth'])))

In [ ]:
if 0: 
    import cartopy
    import cartopy.crs as ccrs
    from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
    
    fig = plt.figure(figsize=(20, 14), dpi=300)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.stock_img()
    
    if 'map_limits' in model.__dict__:
        limits = model.map_limits
        tickspacing = model.map_tick_spacing
    else:
        limits = [min(catalogue.data['longitude']), min(catalogue.data['latitude']), 
                  max(catalogue.data['longitude']), max(catalogue.data['latitude'])]
        if limits[2]-limits[0] > 20:
            tickspacing = 10
        else:
            tickspacing = 5
    
    ax.coastlines()
    # ax.add_feature(cartopy.feature.OCEAN, zorder=0)
    # set the area for the plot
    ax.set_extent([limits[0], limits[2], limits[1], limits[3]], ccrs.Geodetic())
    
    xlo = numpy.floor(limits[0]/10)*10
    ylo = numpy.round(limits[1]/10)*10
    ax.set_xticks(numpy.arange(xlo, limits[2], tickspacing), crs=ccrs.PlateCarree())
    ax.set_yticks(numpy.arange(ylo, limits[3], tickspacing), crs=ccrs.PlateCarree())
    
    idx = numpy.nonzero(flag!=0)

    # catalogue    
    ax.plot(catalogue.data['longitude'], catalogue.data['latitude'], 'o',
            linewidth=6, alpha=0.4, transform=ccrs.Geodetic(), color='blue')

    ax.plot(catalogue_original.data['longitude'][idx], 
            catalogue_original.data['latitude'][idx], 'x',
            linewidth=.5, alpha=0.8, transform=ccrs.Geodetic(), color='red')   

### Declustering

In [ ]:
distance_time_wind = GardnerKnopoffWindow()
config = {'time_distance_window': distance_time_wind, 'fs_time_prop': .9}
declusterer = GardnerKnopoffType1()
vcl, flag = declusterer.decluster(catalogue, config)

catalogue_original = deepcopy(catalogue)
print ('{:s}: {:.2f}'.format('Original number of events'.ljust(text_length), catalogue_original.get_number_events()))
catalogue.select_catalogue_events(numpy.where(flag == 0)[0])
print ('{:s}: {:.2f}'.format('Number of mainshocks'.ljust(text_length), len(catalogue.data['magnitude'])))

### Completeness analysis

In [ ]:
print(min(catalogue.data['year']), max(catalogue.data['year']))
print(min(catalogue.data['month']), max(catalogue.data['month']))
print(min(catalogue.data['day']), max(catalogue.data['day']))

In [ ]:
#
# parameters for completeness analysis
config = {'magnitude_bin': .5, 'time_bin': 10., 'increment_lock': True}
stepp = Stepp1971()
compl_table = stepp.completeness(catalogue, config)
#
# cleaning temporary figures
if os.path.exists('aa'):
    subprocess.call('rm aa', shell=True)
#create_stepp_plot(stepp, 'aa')
if os.path.exists('aa'):
    subprocess.call('rm aa', shell=True)
#
# info
print ('Maximum magnitude is: ', max(catalogue.data['magnitude']))
print ('The catalogue contains %d events' % (catalogue.get_number_events()))
print ('Completeness table: \n')
#
# Display html
for line in compl_table:
    print ('%.2f, %.2f' % (line[0], line[1]))

In [ ]:
ppp = plot_magnitude_time_density(catalogue, mag_int=0.1, time_int=10, normalisation=True, 
                                  bootstrap=0, completeness=compl_table)

## Magnitude-frequency distribution

In [ ]:
idx = numpy.nonzero(compl_table[:,1] < numpy.max(catalogue.data['magnitude']))
#
# settings for MFD calculation
weichert_config = {'magnitude_interval': 0.1, 'reference_magnitude': 0.0}
weichert = Weichert()
bval_wei, sigmab, aval_wei, sigmaa = weichert.calculate(catalogue, weichert_config, completeness=compl_table)
#
# info
print ('bval: %.3f (sigma=%.3f)' % (bval_wei, sigmab))
print ('aval: %.3f (sigma=%.3f)' % (aval_wei, sigmaa))

In [ ]:
mmax_plot = numpy.amax(catalogue.data['magnitude'])
plot_trunc_gr_model(aval_wei, bval_wei, cutoff_magnitude, mmax_plot, 0.1, catalogue=catalogue, 
                    filename='aa', completeness=compl_table, save=False)

## Project info

In [ ]:
model_lab = re.sub('\s', '_', oqtkp.models[model_id].name).lower()
output_path = os.path.join(oqtkp.directory, '%s_catalogue.pkl' % model_lab)
fou = open(output_path,'wb') 
pickle.dump(catalogue, fou)
fou.close()
print ('Catalogue dataset saved into file: \n%s' % (output_path))

In [ ]:
filename = os.path.join(oqtkp.directory, oqtkp.compl_hdf5_filename)
fhdf5 = h5py.File(filename, 'a')
print ('Updating {:s}'.format(filename))
dataset_name = 'whole_catalogue'
#
# Update/create group
if model_id in fhdf5.keys():
    print ('    Group {:s} exists'.format(model_id))
    grp = fhdf5[model_id]
else:
    print ('    Creating group: %s' % (model_id))
    grp = fhdf5.create_group(model_id)
# Update/create dataset
if dataset_name in grp:
    del fhdf5[model_id][dataset_name]
    print ('    Updating dataset: %s' % (dataset_name))
    dataset = grp.create_dataset(dataset_name, data=compl_table)
else:
    print ('    Creating dataset: %s' % (dataset_name))
    dataset = grp.create_dataset(dataset_name, data=compl_table)
fhdf5.close()

In [ ]:
oqtkp.models[model_id].declustered_catalogue_pickle_filename = os.path.relpath(output_path, oqtkp.directory)
oqtkp.save()

In [ ]:
model = oqtkp.models[model_id]
model.get_info()